In [ ]:
# version 5 (final version) - gradio 設定累積歷史提問與答案

# pip3 install langchain_ollama gradio pypdf chromadb
# ollama pull nomic-embed-text

# pip3 pip install ipywidgets
# jupyter nbextension enable --py widgetsnbextension
# pip3 install -U langchain-ollama

In [23]:
import os, sys
sys.path.append('.')

from langchain_ollama import OllamaLLM
import gradio as gr

# 只使用 Ollama 模型進行問答
def ollama_llm(question):
    formatted_prompt = f"總是用繁體中文回答！\n\nQuestion: {question}"
    llm = OllamaLLM(model="llama3.1:70b", base_url="http://localhost:11434")
    
    try:
        response = llm.generate(prompts=[formatted_prompt])
        return response.generations[0][0].text
    except Exception as e:
        return f"An error occurred: {str(e)}"

# 定義 Gradio 介面
def get_important_facts(question):
    return ollama_llm(question)

# 創建 Gradio 應用介面
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="請輸入您的問題"),
  outputs="text",
  title="Ollama Chat",
  description="使用 Llama3 模型直接回答您的問題",
)

# 啟動 Gradio 應用
iface.launch()